In [1]:
import cvxpy as cp
import numpy as np

In [2]:
# n = 3
# p = 3
# np.random.seed(1)
# C = np.random.randn(n, n)
# A = []
# b = []
# for i in range(p):
#     A.append(cp.Parameter((n,n), value=np.random.randn(n, n)))
# #     A.append(np.random.randn(n, n))
#     b.append(np.random.randn())

# # Define and solve the CVXPY problem.
# # Create a symmetric matrix variable.
# X = cp.Variable((n,n), PSD=True)
# # The operator >> denotes matrix inequality.
# # constraints = [X >> 0]
# constraints = []
# constraints += [
#     cp.trace(A[i] @ X) == b[i] for i in range(p)
# ]
# prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
#                   constraints)
# prob.solve()

# # Print result.
# print("The optimal value is", prob.value)
# print("A solution X is")
# print(X.value)

In [141]:
n = 3
# C = np.random.randn(n, n)
c = cp.Parameter((n,1), value=np.array([1,1,1]).reshape((-1,1)), nonneg=True)

A = cp.Parameter((n,n), value=np.array([[-0.99639688, -0.2216588 ,  1.06555159],
       [ 2.00884919,  0.01563207, -0.36271868],
       [-0.9113235 , -0.12149785, -0.24807337]]))
B = cp.Parameter((n,n), value=np.array([[-1.42367144, -0.62673584, -0.45906866],
       [ 0.6591621 ,  0.96948298,  1.1490893 ],
       [ 0.70077944, -1.33992307, -0.17423502]]))
b = cp.Parameter((n,1), value=np.array([[ 1.72769017],
       [-0.97577213],
       [ 1.02798925]]))

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), hermitian=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    A[i][0] * cp.real(X[i][0]) + B[i][0] * cp.imag(X[i][0]) <= b[i] for i in range(n)
]
constraints += [cp.real(X) <= np.ones((n,n))]
# prob = cp.Problem(cp.Minimize(cp.trace(C @ cp.real(X))), constraints)
prob = cp.Problem(cp.Minimize(cp.sum(
    [c[i]*cp.square(cp.real(X[i][1])) + c[i]*cp.real(X[i][2]) for i in range(n)]
)), constraints)
# prob.is_dcp()

prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 0.05107045673607016
A solution X is
[[ 0.99999999+0.j         -0.32148496+0.50057086j -0.34312057-0.24426422j]
 [-0.32148496-0.50057086j  0.35392377+0.j         -0.01196345+0.25028344j]
 [-0.34312057+0.24426422j -0.01196345-0.25028344j  0.17739675+0.j        ]]


In [140]:
n = 3
c = cp.Parameter((n,1), value=np.array([1,1,1]).reshape((-1,1)), nonneg=True)

A = cp.Parameter((n,n), value=np.array([[-0.99639688, -0.2216588 ,  1.06555159],
       [ 2.00884919,  0.01563207, -0.36271868],
       [-0.9113235 , -0.12149785, -0.24807337]]))
B = cp.Parameter((n,n), value=np.array([[-1.42367144, -0.62673584, -0.45906866],
       [ 0.6591621 ,  0.96948298,  1.1490893 ],
       [ 0.70077944, -1.33992307, -0.17423502]]))
b = cp.Parameter((n,1), value=np.array([[ 1.72769017],
       [-0.97577213],
       [ 1.02798925]]))

# Create a symmetric matrix variable.
X = cp.Variable((n,n), hermitian=True)

constraints = [ cp.quad_over_lin(cp.norm(X[i][j]), cp.real(X[i][i])) <= cp.real(X[j][j]) for i,j in [(0,1), (0,2), (1,2)]]
constraints += [
    A[i][0] * cp.real(X[i][0]) + B[i][0] * cp.imag(X[i][0]) <= b[i] for i in range(n)
]
constraints += [cp.real(X) <= np.ones((n,n))]

prob = cp.Problem(cp.Minimize(cp.sum(
    [c[i]*cp.square(cp.real(X[i][1])) + c[i]*cp.real(X[i][2]) for i in range(n)]
)), constraints)

print(prob.is_dcp())

prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

True
The optimal value is -0.2507304200506858
A solution X is
[[ 0.99999999+0.00000000e+00j -0.35074946+4.11384939e-01j
  -0.59609004-5.16125480e-06j]
 [-0.35074946-4.11384939e-01j  0.29226277+0.00000000e+00j
  -0.3222542 +0.00000000e+00j]
 [-0.59609004+5.16125480e-06j -0.3222542 +0.00000000e+00j
   0.35532334+0.00000000e+00j]]


In [119]:
X = cp.Variable((2,2), hermitian=True)
# z^2/y <= x
constraints = [cp.quad_over_lin(cp.norm(X[0][1]), cp.real(X[1][1])) <= cp.real(X[0][0]), cp.real(X[1][1]) <= 2, cp.real(X[0][1]) >= 1 ]
prob = cp.Problem(cp.Minimize(cp.real(X[0][0])), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.5000000046832747

In [53]:
x = cp.Variable()
y = cp.Variable()
z = cp.Variable()
# z^2/y <= x
constraints = [cp.quad_over_lin(z, y) <= x, y <= 2, z >= 1 ]

prob = cp.Problem(cp.Minimize(x), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.5000000136111296

In [81]:
xyz = cp.Variable(shape=(3,))  # first two coordinates are "x" and "y"
# xy >= z^2
t = cp.Variable()
aux_constraint = t >= cp.norm(xyz[2])
lmi_constraint = cp.bmat([[xyz[0], t],[t, xyz[1]]]) >> 0
constraints = [aux_constraint, lmi_constraint, xyz[1] <=2, xyz[2] >= 1]

prob = cp.Problem(cp.Minimize(xyz[0]), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.49999999981725113